In [ ]:
import pandas as pd 
import os
import numpy as np
import sys
import matplotlib 
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import seaborn as sns
import ast
import spacy
from collections import Counter
from itertools import combinations

trainMan ist für das manuelle Traninieren auf eine csv. Dabei kann ausgewählt werden, welches der Hyperparameter optimiert werden soll. 
Es gibt auch die Option mit train alle zur Verfügung stehndende Parameter zu optimieren

In [5]:
class trainMan:
    def __init__(self, path, 
                 max_depthVal=None, max_depthMax=200, max_depthMin=0, 
                 min_sample_splitVal = 2, min_sample_splitMax=10, min_sample_splitMin=2, 
                 clf=None, criterionVal="entropy"):
        self.path = path
        self.trainList = self.__data()
        self.max_depthVal = max_depthVal
        self.max_depthMax = max_depthMax
        self.max_depthMin=max_depthMin
        self.min_sample_splitMax = min_sample_splitMax
        self.min_sample_splitMin = min_sample_splitMin
        self.min_sample_splitVal = min_sample_splitVal
        self.clf = clf
        self.criterionVal = criterionVal
        self.results = np.empty((3, max_depthMax-max_depthMin, min_sample_splitMax-min_sample_splitMin, 3))
    def __data(self):
        self.data = pd.read_csv(self.path)
        columnsLength = self.data.shape[1]-1
        X = self.data.iloc[:,0:columnsLength]
        y = self.data[["groupID"]].values.ravel()
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        return [self.X_train, self.X_test, self.y_train, self.y_test]
    def get_X_train(self):
        return self.X_train
    def out(self):
        if(self.clf == None):
            print("Modell ohne Parameter wird angelegt, da keines vorhanden:")
            self.clf = DecisionTreeClassifier(random_state=42)
            self.clf = self.clf.fit(self.trainList[0], self.trainList[2])
        predict = self.clf.predict(self.trainList[1])
        report = classification_report(self.trainList[3], predict, target_names=["Atheism","Graphics", "Space", "Religion"])
        #print(report)
        #print("F-Score: ",f1_score(predict, self.trainList[3], average='macro'))
        #print("Precision: ",precision_score(predict, self.trainList[3], average='macro'))
        #print("Recall: ",recall_score(predict, self.trainList[3], average='macro'))
        return f1_score(predict, self.trainList[3], average='macro'), precision_score(predict, self.trainList[3], average='macro'), recall_score(predict, self.trainList[3], average='macro')
    def max_depth(self, min, max):
        self.max_depthMax = max
        self.max_depthMin = min
        f = 0
        for i in range(self.max_depthMax):
            if(i<self.max_depthMin):
                continue
            clf = DecisionTreeClassifier(random_state=42, criterion=self.criterionVal, max_depth=i, min_samples_split=self.min_sample_splitVal)
            clf = clf.fit(self.trainList[0], self.trainList[2])
            predict = clf.predict(self.trainList[1])
            if f1_score(predict, self.trainList[3], average='macro')>f:
                f = f1_score(predict, self.trainList[3], average='macro')
                self.max_depthVal = i
                self.clf =  DecisionTreeClassifier(random_state=42, criterion=self.criterionVal, max_depth=i, min_samples_split=self.min_sample_splitVal)
                self.clf = self.clf.fit(self.trainList[0], self.trainList[2])      
        return self.max_depthVal
    def tree(self):
            self.clf = DecisionTreeClassifier(random_state=42, criterion=self.criterionVal, max_depth=self.max_depthVal)
            self.clf = self.clf.fit(self.trainList[0], self.trainList[2])  
            return self.clf  
    def random_state(self, max, min):
        f = 0
        random = None
        for i in range(max):
            if(i<min):
                continue
            clf = DecisionTreeClassifier(random_state=i, criterion=self.criterionVal, max_depth=85, min_samples_split=self.min_sample_splitVal)
            clf = clf.fit(self.trainList[0], self.trainList[2])
            predict = clf.predict(self.trainList[1])
            if f1_score(predict, self.trainList[3], average='macro')>f:
                f = f1_score(predict, self.trainList[3], average='macro')
                random = i
                self.clf =  DecisionTreeClassifier(random_state=random, criterion=self.criterionVal, max_depth=93, min_samples_split=self.min_sample_splitVal)
                self.clf = self.clf.fit(self.trainList[0], self.trainList[2])      
        return random
    def min_sample_split(self):
        f = 0
        for i in range(self.min_sample_splitMax):
            if(i<self.min_sample_splitMin):
                continue
            clf = DecisionTreeClassifier(random_state=42, criterion=self.criterionVal, min_samples_split=i, max_depth=self.max_depthVal)
            clf = clf.fit(self.trainList[0], self.trainList[2])
            predict = clf.predict(self.trainList[1])
            if f1_score(predict, self.trainList[3], average='macro')>f:
                f = f1_score(predict, self.trainList[3], average='macro')
                self.min_sample_splitVal = i    
        return self.min_sample_splitVal
    def criterion(self):
        criterionList = {"entropy", "log_loss", "gini"}
        f = 0
        for criterion in criterionList:
            clf = DecisionTreeClassifier(random_state=42, criterion=criterion, min_samples_split=self.min_sample_splitVal, max_depth=self.max_depthVal)
            clf = clf.fit(self.trainList[0], self.trainList[2])
            predict = clf.predict(self.trainList[1])
            if f1_score(predict, self.trainList[3], average='macro')>f:
                f = f1_score(predict, self.trainList[3], average='macro')
                self.criterionVal = criterion
                self.clf =  DecisionTreeClassifier(random_state=42, criterion=criterion, min_samples_split=self.min_sample_splitVal, max_depth=self.max_depthVal)
                self.clf = self.clf.fit(self.trainList[0], self.trainList[2])
        return self.criterionVal  
    def train(self):
        criterionList = {"entropy", "log_loss", "gini"}
        for criterion in criterionList:
            if (criterion == "entropy"):
                index = 0
            elif (criterion=="log_loss"):
                index = 1
            else:
                index = 2
            for depth in range(self.max_depthMax):
                if(depth < self.max_depthMin):
                    continue
                for min_split in range (self.min_sample_splitMax):
                    if (min_split<self.min_sample_splitMin):
                        continue
                    clf = DecisionTreeClassifier(random_state=42, criterion=criterion, min_samples_split=min_split, max_depth=depth)
                    clf = clf.fit(self.trainList[0], self.trainList[2])
                    predict = clf.predict(self.trainList[1])
                    print(str(index)+"|"+str(depth)+"|"+str(min_split))
                    self.results[index, depth-self.max_depthMin, min_split-self.min_sample_splitMin, 0] = f1_score(predict, self.trainList[3], average='macro')
                    self.results[index, depth-self.max_depthMin, min_split-self.min_sample_splitMin, 1] = precision_score(predict, self.trainList[3], average='macro')
                    self.results[index, depth-self.max_depthMin, min_split-self.min_sample_splitMin, 2] = recall_score(predict, self.trainList[3], average='macro')
        index = np.unravel_index(np.argmax(self.results[:, :, :, 0]), self.results[:, :, :, 0].shape)
        if (index[0] == 0):
            self.criterionVal = "entropy"
        elif (index[0]==1):
            self.criterionVal = "log_loss"
        else:
            self.criterionVal = "gini"
        self.max_depthVal = index[1]+self.max_depthMin
        self.min_sample_splitVal = index[2]+self.min_sample_splitMin
        print(self.criterionVal+"|"+str(self.max_depthVal)+"|"+str(self.min_sample_splitVal))

trainAuto optimiert nur auf max_depth, dafür auf meherere Dateein, dabei kann dann am Besten der vielversprechneste Ansatz herausgefunden werden

In [6]:
class trainAuto:
    def __init__(self, path):
        self.paths = [os.path.relpath(os.path.join(path, datei)) for datei in os.listdir(path) if os.path.isfile(os.path.join(path, datei))]
        self.data = pd.DataFrame(columns=["Datei", "criterion", "max_depth", "min_sample_split", "f_score", "precision", "recall"])
    
    def __save(self):
        self.data.to_csv("trainAuto3.csv", index=False, encoding="utf-8")
        print("\nData gesichert")
    
    def train(self):
        gesamt = len(self.paths)
        
        for i, path in enumerate(self.paths, start=1):
            prozent = (i / gesamt) * 100
            länge = 50
            balken_länge = int(länge * i // gesamt)
            balken = f"[{'#' * balken_länge}{'.' * (länge - balken_länge)}]"
            sys.stdout.write(f"\r{balken} {prozent:.2f}% ({i}/{gesamt})")
            sys.stdout.flush()
            
            tree = trainMan(path=path, criterionVal="entropy")
            max_depth = tree.max_depth(50, 150)
            f_score, precision, recall = tree.out()
            
            self.data = pd.concat([self.data, pd.DataFrame({
                "Datei": [path], 
                "criterion": ["entropy"], 
                "max_depth": [max_depth], 
                "min_sample_split": [2], 
                "f_score": [f_score], 
                "precision": [precision], 
                "recall": [recall]
            })])
        
        self.__save()

trainToTrain speichert immer die von einem trainierten DecissionTree verwendeten Feature ab, um in einem kombinierten DecisionTree alle Feature die schon mal verwendet wurden zu verwenden.
wichtig: immer die gleichen Train/Test Splits
- Optionen:
+ testen mit Ausgabe, kein Schreiben: test
+ in neue Datei schreiben: newOut
+ in default Datei schreiben (nur wenn Ansatz auch gut war): oldOut
+ trainieren auf Basis der neuen Feature: train

In [ ]:
class trainToTrain:
    def __init__(self, pathIn="None", pathOut="test", threshold = 0, paths="toTrain", max_depths=[]):
        self.pathOut = pathOut
        self.pathIn = pathIn
        self.treshold = threshold
        self.paths = [os.path.relpath(os.path.join(paths, datei)) for datei in os.listdir(paths) if os.path.isfile(os.path.join(paths, datei))]
        self.max_depths = max_depths
        self.data = pd.DataFrame(columns=["Datei", "FeatureList", "f_score", "precision", ])
    def test(self):
        tree = trainMan(self.pathIn)
        tree.max_depth(50, 60)
        clf = tree.clf
        self.__importance(clf, tree.get_X_train)
        return self.__getList(clf, tree.get_X_train)
    def newOut(self, outPath):
        if (self.max_depths == []):
            for path in self.paths:
                tree = trainMan(path)
                depth = tree.max_depth(50, 60)
                self.max_depths.append(depth)
        i=0
        for path in self.paths:
            tree = trainMan(path, max_depthVal=self.max_depths[i])
            tree.tree()
            i+=1
            self.data = pd.concat([self.data, pd.DataFrame({
                "Datei": [path],
                "FeatureList": [self.__getList(tree.clf, tree.get_X_train)]
            })])
        self.__save(path=outPath)
    def oldOut(self):
        if (self.max_depths == []):
            for path in self.paths:
                tree = trainMan(path)
                depth = tree.max_depth(50, 200)
                self.max_depths.append(depth)
        i=0
        for path in self.paths:
            tree = trainMan(path, max_depthVal=self.max_depths[i])
            tree.tree()
            f_score, precision, recall = tree.out()
            i+=1
            self.data = pd.concat([self.data, pd.DataFrame({
                "Datei": [path],
                "FeatureList": [self.__getList(tree.clf, tree.get_X_train)], 
                "f_score": [f_score],
                "precision": [precision],
                "recall": [recall]
            })])
        self.__save()
    def __save(self, path="ImportanceTrain.csv"):
        self.data.to_csv(path, index=False, encoding="utf-8")
        print("\nData gesichert")
    def __importance(self, clf, X_train):
        importances = clf.feature_importances_
        feature_importances_df = pd.DataFrame({
            'Feature': X_train().columns,
            'Importance': importances
        }).sort_values(by='Importance', ascending=False)
        non_zero_importance = feature_importances_df[feature_importances_df['Importance'] > self.treshold]
        count_non_zero_importance = non_zero_importance.shape[0]
        print("Anzahl der Zeilen mit Importanzwerten über "+str(self.treshold)+":", count_non_zero_importance)
        top_features = non_zero_importance.head(count_non_zero_importance)
        plt.figure(figsize=(10, 40))
        sns.barplot(data=top_features, x="Importance", y="Feature", palette="viridis")
        plt.title("Top Feature Importances")
        plt.xlabel("Importance Score")
        plt.ylabel("Feature")
        plt.tight_layout()
        plt.show()
    def __getList(self, clf, X_train):
        importances = clf.feature_importances_
        feature_importances_df = pd.DataFrame({
            'Feature': X_train().columns,
            'Importance': importances
        }).sort_values(by='Importance', ascending=False)
        non_zero_importance = feature_importances_df[feature_importances_df['Importance'] > self.treshold]
        count_non_zero_importance = non_zero_importance.shape[0]
        top_features = non_zero_importance.head(count_non_zero_importance)
        return top_features["Feature"].to_list()
    def train(self, inPath = "ImportanceTrain.csv"):
        df = pd.read_csv(inPath)
        df['FeatureList'] = df['FeatureList'].apply(ast.literal_eval) #ast.literal_aval wandelt eine String Liste in eine echte Liste um

        datei_feature_liste = df[['Datei', 'FeatureList']].values.tolist() #wandelt DataFrame Zeilen in echte Listen um

        erste_features = set(df['FeatureList'].iloc[0])
        vergleich = []

        for _, features in df.iterrows():
            aktuelle_features = set(features['FeatureList'])
            
            neue_features = aktuelle_features - erste_features
            fehlende_features = erste_features - aktuelle_features 
            
            vergleich.append({
                'Datei': features['Datei'],
                'Neue Features': len(neue_features),
                'Fehlende Features': len(fehlende_features)
            })

        vergleich_df = pd.DataFrame(vergleich)

        for datei, features in datei_feature_liste:
            print(f"Datei: {datei}\nFeatureList: {features}\n")
        print("\nVergleich:")
        print(vergleich_df)

        kombinierte_features = set()
        for _, features in datei_feature_liste:
            kombinierte_features.update(features)

        print("\nKombinierte Features ohne Dopplung:")
        print(kombinierte_features)
        print(f"\nAnzahl der kombinierten Features: {len(kombinierte_features)}")
        self.__trainCSV(columns = kombinierte_features)
    def __trainCSV(self, columns, out_csv="combinedFeatures.csv"):
        data = pd.read_csv("20 newsgroups/20 newsgroups.csv", on_bad_lines='skip', sep=';')
        data.dropna()
        data = data.iloc[:, 1:3]

        nlp = spacy.load("en_core_web_sm")
        columns = list(columns)
        columns.append("groupID")
        newFeature = pd.DataFrame(columns=columns)
        newFeature.to_csv(out_csv)
        i = 0
        for index, row in data.iterrows():
            text = str(row['text'])
            group = str(row['group'])
            doc = nlp(text)
            count_in_top_1000 = Counter(token.text.lower() for token in doc if token.is_alpha)
            haeufigkeiten = {word: count_in_top_1000.get(word, 0) for word in columns}
            haeufigkeiten['groupID'] = group
            neue_zeile_df = pd.DataFrame([haeufigkeiten])
            neue_zeile_df.reset_index(drop=True, inplace=True)
            newFeature = pd.concat([newFeature, neue_zeile_df], ignore_index=True)
        newFeature.to_csv(out_csv, index=False)
        newFeature.head()
    def combine_and_train(self):
        # Jede Zeile (Datei und Feature-Liste) auslesen
        df = pd.read_csv("ImportanceTrain.csv")
        df['FeatureList'] = df['FeatureList'].apply(ast.literal_eval)  # String-Liste in echte Liste umwandeln
        datei_feature_liste = df[['Datei', 'FeatureList']].values.tolist()

        # Alle Kombinationen aus den Listen erstellen
        for r in range(1, len(datei_feature_liste) + 1):
            for kombination in combinations(datei_feature_liste, r):
                kombinierte_features = set()
                dateinamen = []

                # Features aus der Kombination sammeln
                for datei, features in kombination:
                    kombinierte_features.update(features)
                    dateinamen.append(os.path.basename(datei).replace(".csv", ""))

                # Neuen CSV-Namen erstellen basierend auf der Kombination
                neuer_csv_name = "_".join(dateinamen) + ".csv"

                # TrainCSV mit kombinierten Features aufrufen
                self.__trainCSV(columns=kombinierte_features, out_csv=neuer_csv_name)


        
        


In [22]:
test2 = trainToTrain()
#test2.oldOut()
test2.train()

Datei: toTrain\20newsgroups_processed_1000.csv
FeatureList: ['space', 'graphics', 'god', 'thanks', 'orbit', 'religion', 'file', 'bible', 'hi', 'files', 'sky', 'nasa', 'evidence', 'images', 'objective', 'kent', 'people', 'launch', 'disk', 'moon', 'spacecraft', 'jesus', 'islam', 'moral', 'atheist', 'religious', 'image', 'koresh', 'book', 'need', 'card', 'solar', 'program', 'know', 'yes', 'think', 'color', 'atheists', 'time', 'children', 'little', 'islamic', 'algorithm', 'available', 'christian', 'looking', 'points', 'right', 'actually', 'probably', 'y', 'wrong', 'said', 'idea', 'number', 'package', 'christ', 'certainly', 'past', 'flight', 'version', 'use', 'hear', 'remember', 'like', 'truth', 'good', 'sure', 'gamma', 'going', 'different', 'true', 'attempt', 'carry', 'high', 'write', 'agree', 'want', 'system', 'explain', 'able', 'ask', 'groups', 'believe', 'near', 'got', 'claim', 'care', 'order', 'beliefs', 'comes', 'heard', 'low', 'oh', 'guess', 'info', 'posts', 'post', 'makes', 'fire', 

In [24]:
new = trainMan(path="combinedFeatures.csv")
print(new.max_depth(1, 200))
new.out()

68


(0.6182263247464747, 0.6148219505969806, 0.6574136289648552)

Ideen:
- manuelles Auswählen indem man durch die csv gehen kann und immer sagen ob man die Feature Liste will oder nicht
- automatisches Optimieren duch alle Kombinationen an Feature Listen
- Durch die Urpsurngsdatei gehen und schauen wie viele Feature auf die jeweilige Zeile zutreffen, wenn es in einer Zeile zu wenig sind wird diese vermerkt um die Feature Auswahl zu optimieren

Data processor: 
- notwendiger Schritt beim Arbeiten mit einen kombinierten Feature Liste, es muss sicher gestellt werden das keine Testdaten schon für das Training verwendet werden.
- gibt 3 csv Datein zurück: die Ursprüngliche, die Trainingsdaten und die Testdaten